In [1]:
import pandas as pd
from scorecard import Scorecard

aesthetic evaluation data

In [ ]:
# load data
aesthetic_evaluation_data = pd.read_csv('datasets/aesthetic_evaluation_data.csv')

# get data from author = cmavioso
aesthetic_evaluation_data_cmavioso = aesthetic_evaluation_data[aesthetic_evaluation_data['Author'] == 'cmavioso']
aesthetic_evaluation_data_cmavioso_objective = aesthetic_evaluation_data_cmavioso['Objective Evaluation']
aesthetic_evaluation_data_cmavioso = aesthetic_evaluation_data_cmavioso.drop(columns=['Image Filename','Author','Objective Evaluation', 'sX2L Value','sX2a Value','sX2b Value','sX2Lab Value','sEMDL Value','sEMDa Value','sEMDb Value','sEMDLab Value'])
aesthetic_evaluation_data_cmavioso_X = aesthetic_evaluation_data_cmavioso.drop(columns='Subjective Evaluation')
aesthetic_evaluation_data_cmavioso_y = aesthetic_evaluation_data_cmavioso['Subjective Evaluation']


aesthetic_evaluation_data = aesthetic_evaluation_data.drop(columns=['Image Filename','Author','Objective Evaluation', 'sX2L Value','sX2a Value','sX2b Value','sX2Lab Value','sEMDL Value','sEMDa Value','sEMDb Value','sEMDLab Value'])
aesthetic_evaluation_X = aesthetic_evaluation_data.drop(columns='Subjective Evaluation')
aesthetic_evaluation_y = aesthetic_evaluation_data['Subjective Evaluation']

# map target classes
mapping = {1: 'Poor', 2: 'Fair', 3: 'Good', 4: 'Excellent'}

# get categorical columns
categorical_columns = aesthetic_evaluation_X.select_dtypes(include=['object']).columns.tolist()
print(f'Categorical columns: {categorical_columns}')

# encode categorical columns
for col in categorical_columns:
    aesthetic_evaluation_X[col] = aesthetic_evaluation_X[col].astype('category').cat.codes.astype('int')
    print(f"Column {col} encoded as: {aesthetic_evaluation_X[col].unique()}")


Categorical columns: []


In [ ]:
scorecard = Scorecard()
_,_ = scorecard.fit(aesthetic_evaluation_X, aesthetic_evaluation_y, categorical_columns,
                thresholds_method='CAIM',
                encoding_method='DIFF_CODING',
                model_method='RiskSLIM',
                use_sbc=True,
                mapping=mapping,
                num_nonzero_weights=None,
                show_prints=True,
                params=None,
                file_name='aesthetic_caim_diffcod.csv'
              )

SBC reduction of train set
number of features:  22
original num target classes:  4
original num observations:  1018
using provided mapping:  {1: 'Poor', 2: 'Fair', 3: 'Good', 4: 'Excellent'}
new num features:  24  (original num features + 2 )
new num target classes:  2
new num observations:  3054  (original num observations * 3 )
   BRA Value  LBC Value  UNR Value  BCE Value  BCD Value  BAD Value  \
0   2.940945   0.863337   2.235024   1.371687   6.298452  61.492602   
1   2.940945   0.863337   2.235024   1.371687   6.298452  61.492602   
2   2.940945   0.863337   2.235024   1.371687   6.298452  61.492602   
3   2.037527   2.476401   2.026023   0.450378   2.747946  27.565989   
4   2.037527   2.476401   2.026023   0.450378   2.747946  27.565989   

   BOD Value  pBRA Value  pLBC Value  pUNR Value  ...  cX2a Value  cX2b Value  \
0  92.020190    0.114506    0.032244    0.102389  ...    0.023480    0.152833   
1  92.020190    0.114506    0.032244    0.102389  ...    0.023480    0.152833  

In [4]:
scorecard = Scorecard()
_,_ = scorecard.fit(aesthetic_evaluation_data_cmavioso_X, aesthetic_evaluation_data_cmavioso_y, categorical_columns,
              thresholds_method='CAIM',
              encoding_method='DIFF_CODING',
              model_method='RiskSLIM',
              use_sbc=True,    
              mapping=mapping,            
              )

SBC reduction of train set
number of features:  22
original num target classes:  4
original num observations:  177
using provided mapping:  {1: 'Poor', 2: 'Fair', 3: 'Good', 4: 'Excellent'}
new num features:  24  (original num features + 2 )
new num target classes:  2
new num observations:  531  (original num observations * 3 )
   BRA Value  LBC Value  UNR Value  BCE Value  BCD Value  BAD Value  \
0   0.665006   1.364289   0.656925   0.707364   1.510404   8.084573   
1   0.665006   1.364289   0.656925   0.707364   1.510404   8.084573   
2   0.665006   1.364289   0.656925   0.707364   1.510404   8.084573   
3   3.341076   0.624909   0.219451   0.844360   1.395453  10.079349   
4   3.341076   0.624909   0.219451   0.844360   1.395453  10.079349   

   BOD Value  pBRA Value  pLBC Value  pUNR Value  ...  cX2a Value  cX2b Value  \
0  32.536978    0.027900    0.054852    0.032040  ...    0.056986    0.053987   
1  32.536978    0.027900    0.054852    0.032040  ...    0.056986    0.053987   


In [5]:
# solution given by RiskSLIM with 
# - integer coefs between -5 and 5
# - max model size of 5
# (first value is the intercept)
'''
CPLEX Error  1217: No solution exists.
+---------------------------------------------+-------------------+-----------+
| Pr(Y = +1) = 1.0/(1.0 + exp(-(1 + score))   |                   |           |
| =========================================== | ================= | ========= |
| feat13-bin3                                 |          5 points |   + ..... |
| feat5-bin2                                  |          2 points |   + ..... |
| feat1-bin2                                  |         -1 points |   + ..... |
| feat8-bin2                                  |         -1 points |   + ..... |
| feat11-bin2                                 |         -5 points |   + ..... |
| =========================================== | ================= | ========= |
| ADD POINTS FROM ROWS 1 to 5                 |             SCORE |   = ..... |
+---------------------------------------------+-------------------+-----------+
 'loss_value': inf,
 'objective_value': inf,
 'optimality_gap': inf,
 'solution': array([ 1.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
       -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -5.,  0.,  0.,  0.,
        0.,  0.,  0.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0., -0., -0., -0.]),
'''

"\nCPLEX Error  1217: No solution exists.\n+---------------------------------------------+-------------------+-----------+\n| Pr(Y = +1) = 1.0/(1.0 + exp(-(1 + score))   |                   |           |\n| =========================================== | ================= | ========= |\n| feat13-bin3                                 |          5 points |   + ..... |\n| feat5-bin2                                  |          2 points |   + ..... |\n| feat1-bin2                                  |         -1 points |   + ..... |\n| feat8-bin2                                  |         -1 points |   + ..... |\n| feat11-bin2                                 |         -5 points |   + ..... |\n| =========================================== | ================= | ========= |\n| ADD POINTS FROM ROWS 1 to 5                 |             SCORE |   = ..... |\n+---------------------------------------------+-------------------+-----------+\n 'loss_value': inf,\n 'objective_value': inf,\n 'optimality_gap': 

In [ ]:
# solution given by RiskSLIM with 
# - integer coefs between -5 and 5 (more than this is not supported by CPLEX free version)
# - max model size of 10

'''
CPLEX Error  1217: No solution exists.
+---------------------------------------------+-------------------+-----------+
| Pr(Y = +1) = 1.0/(1.0 + exp(-(1 + score))   |                   |           |
| =========================================== | ================= | ========= |
| feat19-bin3                                 |          5 points |   + ..... |
| feat20-bin3                                 |          5 points |   + ..... |
| feat2-bin2                                  |          4 points |   + ..... |
| feat12-bin2                                 |         -2 points |   + ..... |
| feat21-bin2                                 |         -2 points |   + ..... |
| feat10-bin2                                 |         -3 points |   + ..... |
| feat5-bin3                                  |         -5 points |   + ..... |
| feat9-bin2                                  |         -5 points |   + ..... |
| feat12-bin3                                 |         -5 points |   + ..... |
| feat13-bin3                                 |         -5 points |   + ..... |
| =========================================== | ================= | ========= |
| ADD POINTS FROM ROWS 1 to 10                |             SCORE |   = ..... |
+---------------------------------------------+-------------------+-----------+
 'solution': array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0., -5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0., -5.,  0.,  0., -3.,  0.,  0.,  0.,  0.,  0., -2.,
       -5.,  0.,  0., -5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  5.,  0.,  0.,  5.,  0.,
       -2.,  0.,  0.,  0.,  0.]),
'''

"\nCPLEX Error  1217: No solution exists.\n+---------------------------------------------+-------------------+-----------+\n| Pr(Y = +1) = 1.0/(1.0 + exp(-(1 + score))   |                   |           |\n| =========================================== | ================= | ========= |\n| feat19-bin3                                 |          5 points |   + ..... |\n| feat20-bin3                                 |          5 points |   + ..... |\n| feat2-bin2                                  |          4 points |   + ..... |\n| feat12-bin2                                 |         -2 points |   + ..... |\n| feat21-bin2                                 |         -2 points |   + ..... |\n| feat10-bin2                                 |         -3 points |   + ..... |\n| feat5-bin3                                  |         -5 points |   + ..... |\n| feat9-bin2                                  |         -5 points |   + ..... |\n| feat12-bin3                                 |         -5 points 

tentar com datasets mais pequenos (filtrar por autor)!!!!!